In [ ]:
from sklearn import preprocessing
import numpy as np

In [13]:
class txtReader:
    def __init__(self, filename):
        self.filename = filename

    def read_split(self):
        with open(self.filename, 'r') as f:
            file_read = f.read()
        efg = []
        lines = file_read.split('\n')
        efg.append(lines)
        text = []
        text_id = []
        for i in lines:
            if i != '':
                word = i.split('\t')
                text.append(word[0])
                text_id.append(word[1])
        # text = ['_ZERO_'] + text + ['_ZERO_']
        # text_id = ['_ZERO_'] + text_id + ['_ZERO_']
        return text, text_id


In [21]:
class alphabet:
    def __init__(self, filename):
        self.filename = filename
        self.text = []
        self.text_id = []

    def read_split(self):
        txt = txtReader(self.filename)
        text, text_id = txt.read_split()
        self.text = text
        self.text_id = text_id
        return text, text_id
    
    def labelEncoder(self):
        text, text_id = self.read_split()
        le = preprocessing.LabelEncoder()
        text = le.fit_transform(text) + 1
        le_id = preprocessing.LabelEncoder()
        text_id = le_id.fit_transform(text_id) + 1
        text = np.concatenate(([0], text, [0]))
        text_id = np.concatenate(([0], text_id, [0]))
        return text, text_id

In [20]:
train, train_id = alphabet('materiales_practica/datasets/PartTUT/train.txt').labelEncoder()
# train = train.labelEncoder()
print(train_id, 'id')
print(train, 'train')
# Contar cantidad de ceros que hay en train sabiendo que es un array de numpy
print(np.count_nonzero(train == 0), 'cantidad de ceros')

[ 0  8  2 ...  8 13  0] id
[   0  596 4860 ...  723   18    0] train
2 cantidad de ceros


In [43]:
class FFTagger():
    def __init__():
        pass

    def train():
        pass


['NOUN', 'ADP', 'DET', 'NOUN', 'AUX', 'PART', 'VERB', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'NOUN', 'PUNCT', 'PROPN', 'PROPN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'PUNCT', 'ADP', 'PUNCT', 'VERB', 'PUNCT', 'NOUN', 'PUNCT', 'PROPN', 'PROPN', 'VERB', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'VERB', 'PUNCT', 'CCONJ', 'VERB', 'NOUN', 'ADP', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'NOUN', 'CCONJ', 'PUNCT', 'CCONJ', 'ADJ', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'VERB', 'ADV', 'PUNCT', 'PRON', 'VERB', 'CCONJ', 'VERB', 'PART', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'VERB', 'PRON', 'DET', 'NOUN', 'VERB', 'ADV', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'ADJ', 'NOUN', 'AUX', 'PART', 'AUX', 'VERB', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'VERB', 'ADV', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP'